<a href="https://colab.research.google.com/github/444112029012/4112029012_cal/blob/main/model/model_url/XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Colab 進行matplotlib繪圖時顯示繁體中文
# 下載台北思源黑體並命名taipei_sans_tc_beta.ttf，移至指定路徑
!wget -O TaipeiSansTCBeta-Regular.ttf https://drive.google.com/uc?id=1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_&export=download

import matplotlib

# 改style要在改font之前
# plt.style.use('seaborn')

matplotlib.font_manager.fontManager.addfont('TaipeiSansTCBeta-Regular.ttf')
matplotlib.rc('font', family='Taipei Sans TC Beta')

--2025-11-16 07:38:31--  https://drive.google.com/uc?id=1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_
Resolving drive.google.com (drive.google.com)... 108.177.98.101, 108.177.98.100, 108.177.98.138, ...
Connecting to drive.google.com (drive.google.com)|108.177.98.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_ [following]
--2025-11-16 07:38:31--  https://drive.usercontent.google.com/download?id=1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 173.194.202.132, 2607:f8b0:400e:c00::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|173.194.202.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20659344 (20M) [application/octet-stream]
Saving to: ‘TaipeiSansTCBeta-Regular.ttf’

TaipeiSansTCBeta-Re 100%[===================>]  19.70M  67.7MB/s    in 0.3s    

2025-11-16 07:3

In [ ]:
from google.colab import files
import pandas as pd
def load_data() -> pd.DataFrame:
  uploaded = files.upload()

  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))

  # 將上傳的檔案讀取到 pandas DataFrame 中
  # 假設上傳的檔案是 CSV 格式。如果是其他格式，您可能需要調整讀取函數（例如：pd.read_excel）
  file_name = next(iter(uploaded))
  df = pd.read_csv(file_name)

  # 顯示 DataFrame 的前 5 行
  display(df.head())
  return df

In [ ]:
df = load_data()


Saving phishing_dataset_combine2.csv to phishing_dataset_combine2.csv
User uploaded file "phishing_dataset_combine2.csv" with length 21868092 bytes


,url,target,length_url,length_hostname,ip,nb_dots,nb_hyphens,nb_at,nb_qm,nb_and,...,length_words_raw,char_repeat,shortest_word_host,shortest_word_path,longest_words_raw,longest_word_host,longest_word_path,avg_words_raw,avg_word_host,avg_word_path
0,https://www.tinywebgallery.com,1,30,22,0,2,0,0,0,0,...,25,1,3,0,14,14,0,6.25,6.666667,0.0
1,http://www.madangfood.com,0,25,18,0,2,0,0,0,0,...,20,1,3,0,10,10,0,5.00,5.333333,0.0
2,https://www.inference.vc,1,24,16,0,2,0,0,0,0,...,19,1,2,0,9,9,0,4.75,4.666667,0.0
3,https://www.torforgeblog.com,1,28,20,0,2,0,0,0,0,...,23,1,3,0,12,12,0,5.75,6.000000,0.0
4,https://tinyurl.com/y98bytjf,0,28,11,0,1,0,0,0,0,...,23,1,3,8,8,7,8,5.75,5.000000,8.0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4864 entries, 0 to 5181
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   url                    4864 non-null   object 
 1   target                 4864 non-null   int64  
 2   phish_hints            4864 non-null   int64  
 3   domain_in_brand        4864 non-null   int64  
 4   nb_hyperlinks          4864 non-null   int64  
 5   ratio_intHyperlink     4864 non-null   float64
 6   ratio_extHyperlinks    4864 non-null   float64
 7   ratio_extRedirection   4864 non-null   float64
 8   ratio_extErrors        4864 non-null   float64
 9   external_favicon       4864 non-null   int64  
 10  links_in_tags          4864 non-null   int64  
 11  ratio_extMedia         4864 non-null   float64
 12  safe_anchor            4864 non-null   int64  
 13  empty_title            4864 non-null   int64  
 14  domain_in_title        4864 non-null   int64  
 15  domain_wi

In [ ]:
df['feature_extracted'].unique()

array([ 1.,  0., nan])

In [ ]:
df = df[df['feature_extracted'] == 1]

In [ ]:
df.drop('url', axis=1, inplace=True)

In [ ]:
df

,url,target,phish_hints,domain_in_brand,nb_hyperlinks,ratio_intHyperlink,ratio_extHyperlinks,ratio_extRedirection,ratio_extErrors,external_favicon,links_in_tags,ratio_extMedia,safe_anchor,empty_title,domain_in_title,domain_with_copyright,has_meta_refresh,has_js_redirect,feature_extracted
0,http://www.ishanarora.com/2009/07/29/windows-7...,0,0,0,29,0.827586,0.172414,0.0,0.000000,0,51,0.000000,0,0,0,0,0.0,0.0,1.0
1,http://icyte.com/snapshots/show/05deeb7f5c0c21...,1,0,1,0,0.000000,0.000000,0.0,0.000000,0,4,0.000000,1,0,0,0,0.0,0.0,1.0
2,http://ustasb.com/officesnake/,0,0,0,1,0.000000,1.000000,0.0,0.000000,0,10,0.000000,1,0,0,0,0.0,0.0,1.0
3,http://www.dw.com/en/us-and-somali-forces-stri...,0,0,1,74,0.648649,0.351351,0.0,0.027027,0,103,0.000000,0,0,0,1,0.0,0.0,1.0
4,https://tinyurl.com/sdi-template,1,0,1,24,0.958333,0.041667,0.0,0.041667,0,49,0.000000,0,0,0,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5177,http://www.explainthatstuff.com/streamingmedia...,0,0,0,106,0.641509,0.358491,0.0,0.028302,1,132,1.000000,0,0,0,0,0.0,0.0,1.0
5178,https://goiena.eus/,0,0,1,407,0.879607,0.120393,0.0,0.007371,0,574,0.915966,0,0,1,1,0.0,0.0,1.0
5179,http://www.nbc.com/nbc-nightly-news,0,0,1,144,0.888889,0.111111,0.0,0.000000,0,173,0.000000,0,0,1,1,0.0,0.0,1.0
5180,http://www.sandwichmill.net/img/htaccess/secured/,1,1,0,5,0.000000,1.000000,0.0,1.000000,0,14,0.125000,0,0,0,0,0.0,0.0,1.0


In [ ]:
df['target']

,target
0,0
1,1
2,0
3,0
4,1
...,...
5179,0
5180,0
5181,1
5182,0


In [ ]:
df['target'] = pd.get_dummies(df['status'])['phishing'].astype('int')
df.drop('status', axis=1, inplace=True)

In [ ]:
df.drop('url', axis=1, inplace=True)
df.drop('fetch_status', axis=1, inplace=True)
df.drop('summary_of_intent', axis=1, inplace=True)
df = df[df['gemini_status'] == 'OK']
df.drop('gemini_status', axis=1, inplace=True)

/tmp/ipython-input-4242139424.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop('gemini_status', axis=1, inplace=True)


In [ ]:
df.drop('ip', axis=1, inplace=True)

/tmp/ipython-input-4042677366.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop('ip', axis=1, inplace=True)


In [ ]:
df

,creates_urgency,uses_threats,requests_sensitive_info,offers_unrealistic_rewards,has_spelling_grammar_errors,impersonated_brand,language_professionalism_score,overall_phishing_likelihood_score,target
0,False,False,False,False,False,446,10.0,0.0,0
1,False,False,False,False,False,446,10.0,0.0,0
2,False,False,False,False,False,439,9.0,2.0,0
3,False,False,False,False,False,446,7.0,1.0,0
4,False,False,False,False,False,446,9.0,1.0,0
...,...,...,...,...,...,...,...,...,...
5178,False,False,False,False,False,446,7.0,1.0,1
5179,False,False,False,False,False,446,7.0,1.0,0
5180,False,False,False,False,False,446,9.0,1.0,0
5181,False,False,False,False,False,216,10.0,0.0,0


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [ ]:
df['impersonated_brand'] = df['impersonated_brand'].fillna('NA')
le = LabelEncoder()
df['impersonated_brand'] = le.fit_transform(df['impersonated_brand'])

/tmp/ipython-input-3498642987.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['impersonated_brand'] = df['impersonated_brand'].fillna('NA')
/tmp/ipython-input-3498642987.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['impersonated_brand'] = le.fit_transform(df['impersonated_brand'])


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147225 entries, 0 to 147224
Data columns (total 46 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   target              147225 non-null  int64  
 1   length_url          147225 non-null  int64  
 2   length_hostname     147225 non-null  int64  
 3   ip                  147225 non-null  int64  
 4   nb_dots             147225 non-null  int64  
 5   nb_hyphens          147225 non-null  int64  
 6   nb_at               147225 non-null  int64  
 7   nb_qm               147225 non-null  int64  
 8   nb_and              147225 non-null  int64  
 9   nb_or               147225 non-null  int64  
 10  nb_eq               147225 non-null  int64  
 11  nb_underscore       147225 non-null  int64  
 12  nb_tilde            147225 non-null  int64  
 13  nb_percent          147225 non-null  int64  
 14  nb_slash            147225 non-null  int64  
 15  nb_star             147225 non-nul

In [ ]:
df['creates_urgency'] = df['creates_urgency'].astype(bool)
df['uses_threats'] = df['uses_threats'].astype(bool)
df['offers_unrealistic_rewards'] = df['offers_unrealistic_rewards'].astype(bool)
df['requests_sensitive_info'] = df['requests_sensitive_info'].astype(bool)
df['has_spelling_grammar_errors'] = df['has_spelling_grammar_errors'].astype(bool)

/tmp/ipython-input-3925859276.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['creates_urgency'] = df['creates_urgency'].astype(bool)
/tmp/ipython-input-3925859276.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['uses_threats'] = df['uses_threats'].astype(bool)
/tmp/ipython-input-3925859276.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://panda

In [ ]:
df.drop('url', axis=1, inplace=True)

# ***url***

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier # 針對二元分類使用 XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.preprocessing import LabelEncoder # 如果目標變數是字串 'yes'/'no' 等

from google.colab import files

def load_data() -> pd.DataFrame:
  uploaded = files.upload()

  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))

  # 將上傳的檔案讀取到 pandas DataFrame 中
  # 假設上傳的檔案是 CSV 格式。如果是其他格式，您可能需要調整讀取函數（例如：pd.read_excel）
  file_name = next(iter(uploaded))
  df = pd.read_csv(file_name)

  # 顯示 DataFrame 的前 5 行
  display(df.head())
  return df
# df = load_data()

# 定義特徵 (X) 和目標變數 (y)
X = df.drop('target', axis=1) # 假設 'target' 是您的目標欄位
y = df['target']

# 如果目標變數是字串 (例如 'yes', 'no')，需要進行編碼
# le = LabelEncoder()
# y = le.fit_transform(y) # 將 'yes' 轉換為 1, 'no' 轉換為 0 (或反之，取決於排序)

# --- 2. 處理缺失值 (範例，請根據您的資料調整) ---
# 簡單的填補範例 (用中位數填補數值型缺失值)
# for col in X.columns:
#     if X[col].isnull().any():
#         if X[col].dtype in ['int64', 'float64']:
#             X[col].fillna(X[col].median(), inplace=True)
#         else: # 對於類別型，可以用眾數或創建一個 'Missing' 類別
#             X[col].fillna(X[col].mode()[0], inplace=True)

# --- 3. 處理類別特徵 (如果有的話，且未事先處理) ---
# XGBoost 預設處理數值，對於類別特徵需要 One-Hot Encoding
# X = pd.get_dummies(X, columns=['categorical_feature1', 'categorical_feature2'])

# --- 4. 分割資料集 (訓練集和測試集) ---
# test_size=0.2 表示 20% 的資料用於測試，random_state 確保結果可重現
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
# stratify=y 確保訓練集和測試集中目標變數的類別比例與原始資料集相同，這對於不平衡的二元分類數據很重要。

# --- 5. 建立和訓練 XGBoost 模型 ---
# XGBClassifier 常用參數:
#   objective: 'binary:logistic' (輸出機率) 或 'binary:logitraw' (輸出 log-odds)
#   eval_metric: 'logloss', 'auc', 'error' (分類錯誤率), 'rmse' (回歸) 等
#   use_label_encoder: 在新版本中已棄用，通常設為 False 並手動處理標籤。
#   n_estimators: 樹的數量
#   learning_rate: 學習率 (每次迭代的權重縮減)
#   max_depth: 每棵樹的最大深度
#   subsample: 訓練每棵樹時，採樣的樣本比例
#   colsample_bytree: 訓練每棵樹時，採樣的特徵比例
#   gamma: 執行一個分區所需的最小損失減少量
#   reg_alpha (L1 regularization) 和 reg_lambda (L2 regularization)
#   scale_pos_weight: 處理類別不平衡問題，可設定為 (負類樣本數 / 正類樣本數)

model = XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss', # 評估指標，對於二元分類，logloss 或 auc 常用
    n_estimators=100,      # 樹的數量
    learning_rate=0.1,     # 學習率
    max_depth=3,           # 樹的最大深度
    random_state=42,
    use_label_encoder=False # 新版本中這個參數通常設為 False
)

print("開始訓練 XGBoost 模型...")
model.fit(X_train, y_train)
print("模型訓練完成。")

# --- 6. 預測和評估模型 ---
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1] # 獲取預測為正類 (1) 的機率

# 計算評估指標
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

print(f"\n模型評估結果 (測試集):")
print(f"準確度 (Accuracy): {accuracy:.4f}")
print(f"精確度 (Precision): {precision:.4f}")
print(f"召回率 (Recall): {recall:.4f}")
print(f"F1 分數 (F1 Score): {f1:.4f}")
print(f"ROC AUC: {roc_auc:.4f}")

# --- 7. 查看特徵重要性 (XGBoost 內建) ---
feature_importances = model.feature_importances_
feature_names = X.columns

importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

pd.set_option('display.max_rows', None)
print("\n特徵重要性:")
print(importance_df)



開始訓練 XGBoost 模型...


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [07:42:36] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


模型訓練完成。

模型評估結果 (測試集):
準確度 (Accuracy): 0.9663
精確度 (Precision): 0.9904
召回率 (Recall): 0.9492
F1 分數 (F1 Score): 0.9694
ROC AUC: 0.9917

特徵重要性:
               Feature  Importance
24         https_token    0.438589
13            nb_slash    0.337235
20              nb_www    0.039524
38  shortest_word_path    0.018199
44       avg_word_path    0.016332
21              nb_com    0.013055
9                nb_eq    0.012934
31       nb_subdomains    0.011523
7               nb_and    0.009012
1      length_hostname    0.008873
41   longest_word_path    0.008719
0           length_url    0.008552
26   ratio_digits_host    0.007786
30    tld_in_subdomain    0.006622
43       avg_word_host    0.006158
15            nb_colon    0.005884
4           nb_hyphens    0.005880
33       prefix_suffix    0.005569
25    ratio_digits_url    0.004789
37  shortest_word_host    0.004736
39   longest_words_raw    0.004699
22           nb_dslash    0.003465
42       avg_words_raw    0.003352
11            nb_til

In [ ]:
df = load_data()


Saving phishing_dataset_html_combine2.csv to phishing_dataset_html_combine2.csv
User uploaded file "phishing_dataset_html_combine2.csv" with length 12613384 bytes


,url,target,phish_hints,domain_in_brand,nb_hyperlinks,ratio_intHyperlink,ratio_extHyperlinks,ratio_extRedirection,ratio_extErrors,external_favicon,links_in_tags,ratio_extMedia,safe_anchor,empty_title,domain_in_title,domain_with_copyright,has_meta_refresh,has_js_redirect,feature_extracted
0,http://www.ishanarora.com/2009/07/29/windows-7...,0,0,0,29,0.827586,0.172414,0.0,0.000000,0,51,0.0,0,0,0,0,0,0,1
1,http://icyte.com/snapshots/show/05deeb7f5c0c21...,1,0,1,0,0.000000,0.000000,0.0,0.000000,0,4,0.0,1,0,0,0,0,0,1
2,http://ustasb.com/officesnake/,0,0,0,1,0.000000,1.000000,0.0,0.000000,0,10,0.0,1,0,0,0,0,0,1
3,http://www.dw.com/en/us-and-somali-forces-stri...,0,0,1,74,0.648649,0.351351,0.0,0.027027,0,103,0.0,0,0,0,1,0,0,1
4,https://tinyurl.com/sdi-template,1,0,1,24,0.958333,0.041667,0.0,0.041667,0,49,0.0,0,0,0,0,0,0,1


In [ ]:
df.drop('url', axis=1, inplace=True)

In [ ]:
df

KeyboardInterrupt: 

In [ ]:
df['has_meta_refresh'].unique()

array([0, 1])

In [ ]:
Tdf = df[df['feature_extracted'] == 1]
Fdf = df[df['feature_extracted'] == 0]

In [ ]:
df = Tdf.drop('feature_extracted', axis=1)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147225 entries, 0 to 147224
Data columns (total 46 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   target              147225 non-null  int64  
 1   length_url          147225 non-null  int64  
 2   length_hostname     147225 non-null  int64  
 3   ip                  147225 non-null  int64  
 4   nb_dots             147225 non-null  int64  
 5   nb_hyphens          147225 non-null  int64  
 6   nb_at               147225 non-null  int64  
 7   nb_qm               147225 non-null  int64  
 8   nb_and              147225 non-null  int64  
 9   nb_or               147225 non-null  int64  
 10  nb_eq               147225 non-null  int64  
 11  nb_underscore       147225 non-null  int64  
 12  nb_tilde            147225 non-null  int64  
 13  nb_percent          147225 non-null  int64  
 14  nb_slash            147225 non-null  int64  
 15  nb_star             147225 non-nul

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
numerical_cols = ['nb_hyperlinks', 'links_in_tags']
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

KeyError: "None of [Index(['nb_hyperlinks', 'links_in_tags'], dtype='object')] are in the [columns]"

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140978 entries, 0 to 140977
Data columns (total 18 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   target                 140978 non-null  int64  
 1   phish_hints            140978 non-null  int64  
 2   domain_in_brand        140978 non-null  int64  
 3   nb_hyperlinks          140978 non-null  int64  
 4   ratio_intHyperlink     140978 non-null  float64
 5   ratio_extHyperlinks    140978 non-null  float64
 6   ratio_extRedirection   140978 non-null  float64
 7   ratio_extErrors        140978 non-null  float64
 8   external_favicon       140978 non-null  int64  
 9   links_in_tags          140978 non-null  int64  
 10  ratio_extMedia         140978 non-null  float64
 11  safe_anchor            140978 non-null  int64  
 12  empty_title            140978 non-null  int64  
 13  domain_in_title        140978 non-null  int64  
 14  domain_with_copyright  140978 non-nu

In [ ]:
# 將 'target' 欄位從目前的型態轉換成整數 (int)
df['target'] = df['target'].astype(int)
df['phish_hints'] = df['phish_hints'].astype(int)
df['domain_in_brand'] = df['domain_in_brand'].astype(int)
df['nb_hyperlinks'] = df['nb_hyperlinks'].astype(int)
df['has_meta_refresh'] = df['has_meta_refresh'].astype(int)
df['has_js_redirect'] = df['has_js_redirect'].astype(int)
df['external_favicon'] = df['external_favicon'].astype(int)
df['feature_extracted'] = df['feature_extracted'].astype(int)
df['links_in_tags'] = df['links_in_tags'].astype(int)
df['safe_anchor'] = df['safe_anchor'].astype(int)
df['empty_title'] = df['empty_title'].astype(int)
df['domain_in_title'] = df['domain_in_title'].astype(int)
df['domain_with_copyright'] = df['domain_with_copyright'].astype(int)
# 將 'feature1' 欄位轉換成浮點數 (float)
df['ratio_intHyperlink'] = df['ratio_intHyperlink'].astype(float)
df['ratio_extHyperlinks'] = df['ratio_extHyperlinks'].astype(float)
df['ratio_extRedirection'] = df['ratio_extRedirection'].astype(float)
df['ratio_extErrors'] = df['ratio_extErrors'].astype(float)
df['ratio_extMedia'] = df['ratio_extMedia'].astype(float)

KeyError: 'feature_extracted'

In [ ]:
df['has_meta_refresh'].unique()

array([ 0.,  1., nan])

In [ ]:
def save_df(df, filename = 'phishing_dataset_for_trainning'):
    """
    將指定的 DataFrame 存成 CSV 檔案。

    參數:
    - df: 要儲存的 DataFrame
    - filename: 存檔的檔名（例如 'output.csv'）

    功能:
    - 使用 UTF-8 with BOM 編碼避免中文亂碼
    - 不包含索引欄位
    """
    try:
        df.to_csv(filename, index=False, encoding='utf-8-sig')
        print(f"✅ 資料已成功儲存至 '{filename}'")
    except Exception as e:
        print(f"❌ 儲存失敗：{e}")
save_df(df, 'phishing_dataset_html_features.csv')

✅ 資料已成功儲存至 'phishing_dataset_html_features.csv'


In [ ]:
df.drop('url', axis=1, inplace=True)

In [ ]:
df.head()

,target,phish_hints,domain_in_brand,nb_hyperlinks,ratio_intHyperlink,ratio_extHyperlinks,ratio_extRedirection,ratio_extErrors,external_favicon,links_in_tags,ratio_extMedia,safe_anchor,empty_title,domain_in_title,domain_with_copyright,has_meta_refresh,has_js_redirect,feature_extracted
0,0,0,0,29,0.827586,0.172414,0.0,0.000000,0,51,0.0,0,0,0,0,0,0,1
1,1,0,1,0,0.000000,0.000000,0.0,0.000000,0,4,0.0,1,0,0,0,0,0,1
2,0,0,0,1,0.000000,1.000000,0.0,0.000000,0,10,0.0,1,0,0,0,0,0,1
3,0,0,1,74,0.648649,0.351351,0.0,0.027027,0,103,0.0,0,0,0,1,0,0,1
4,1,0,1,24,0.958333,0.041667,0.0,0.041667,0,49,0.0,0,0,0,0,0,0,1


In [ ]:
df = df[df['feature_extracted'] == 1]

In [ ]:
df.drop('feature_extracted', axis=1, inplace=True)

/tmp/ipython-input-1254700529.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop('feature_extracted', axis=1, inplace=True)


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier # 針對二元分類使用 XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.preprocessing import LabelEncoder # 如果目標變數是字串 'yes'/'no' 等
from google.colab import files

from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import BorderlineSMOTE

def load_data() -> pd.DataFrame:
  uploaded = files.upload()

  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))

  # 將上傳的檔案讀取到 pandas DataFrame 中
  # 假設上傳的檔案是 CSV 格式。如果是其他格式，您可能需要調整讀取函數（例如：pd.read_excel）
  file_name = next(iter(uploaded))
  df = pd.read_csv(file_name)

  # 顯示 DataFrame 的前 5 行
  display(df.head())
  return df
# df = load_data()

# 定義特徵 (X) 和目標變數 (y)
X = df.drop('target', axis=1) # 假設 'target' 是您的目標欄位
y = df['target']

# 如果目標變數是字串 (例如 'yes', 'no')，需要進行編碼
# le = LabelEncoder()
# y = le.fit_transform(y) # 將 'yes' 轉換為 1, 'no' 轉換為 0 (或反之，取決於排序)

# --- 2. 處理缺失值 (範例，請根據您的資料調整) ---
# 簡單的填補範例 (用中位數填補數值型缺失值)
# for col in X.columns:
#     if X[col].isnull().any():
#         if X[col].dtype in ['int64', 'float64']:
#             X[col].fillna(X[col].median(), inplace=True)
#         else: # 對於類別型，可以用眾數或創建一個 'Missing' 類別
#             X[col].fillna(X[col].mode()[0], inplace=True)

# --- 3. 處理類別特徵 (如果有的話，且未事先處理) ---
# XGBoost 預設處理數值，對於類別特徵需要 One-Hot Encoding
# X = pd.get_dummies(X, columns=['categorical_feature1', 'categorical_feature2'])

# --- 4. 分割資料集 (訓練集和測試集) ---
# test_size=0.2 表示 20% 的資料用於測試，random_state 確保結果可重現
# X = scaler.fit_transform(X[['links_in_tags', 'nb_hyperlinks', 'ratio_intHyperlink', 'ratio_extHyperlinks', 'ratio_extRedirection',	'ratio_extErrors', 'ratio_extMedia']])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)
# scaler = StandardScaler()

# smote = BorderlineSMOTE(random_state=42)
# X_train, y_train = smote.fit_resample(X_train, y_train)
# stratify=y 確保訓練集和測試集中目標變數的類別比例與原始資料集相同，這對於不平衡的二元分類數據很重要。

# --- 5. 建立和訓練 XGBoost 模型 ---
# XGBClassifier 常用參數:
#   objective: 'binary:logistic' (輸出機率) 或 'binary:logitraw' (輸出 log-odds)
#   eval_metric: 'logloss', 'auc', 'error' (分類錯誤率), 'rmse' (回歸) 等
#   use_label_encoder: 在新版本中已棄用，通常設為 False 並手動處理標籤。
#   n_estimators: 樹的數量
#   learning_rate: 學習率 (每次迭代的權重縮減)
#   max_depth: 每棵樹的最大深度
#   subsample: 訓練每棵樹時，採樣的樣本比例
#   colsample_bytree: 訓練每棵樹時，採樣的特徵比例
#   gamma: 執行一個分區所需的最小損失減少量
#   reg_alpha (L1 regularization) 和 reg_lambda (L2 regularization)
#   scale_pos_weight: 處理類別不平衡問題，可設定為 (負類樣本數 / 正類樣本數)

model = XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss', # 評估指標，對於二元分類，logloss 或 auc 常用
    n_estimators=400,      # 樹的數量
    learning_rate=0.001,     # 學習率
    max_depth=5,           # 樹的最大深度
    random_state=42,
    use_label_encoder=False # 新版本中這個參數通常設為 False
)

print("開始訓練 XGBoost 模型...")
model.fit(X_train, y_train)
print("模型訓練完成。")

# --- 6. 預測和評估模型 ---
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1] # 獲取預測為正類 (1) 的機率

# 計算評估指標
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

print(f"\n模型評估結果 (測試集):")
print(f"準確度 (Accuracy): {accuracy:.4f}")
print(f"精確度 (Precision): {precision:.4f}")
print(f"召回率 (Recall): {recall:.4f}")
print(f"F1 分數 (F1 Score): {f1:.4f}")
print(f"ROC AUC: {roc_auc:.4f}")

# --- 7. 查看特徵重要性 (XGBoost 內建) ---
feature_importances = model.feature_importances_
feature_names = X.columns

importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

pd.set_option('display.max_rows', None)
print("\n特徵重要性:")
print(importance_df)


# --- 8. (可選) 使用 SHAP 進行更深入的特徵重要性分析 ---
# 需要安裝 shap 函式庫: pip install shap
# import shap

# # 建立 SHAP Explainer
# explainer = shap.TreeExplainer(model)
# # 計算 SHAP 值
# shap_values = explainer.shap_values(X) # 輸出為列表，因為二元分類有兩個類別的 SHAP 值
# # 如果是二元分類，通常我們關心的是正類的 SHAP 值 (索引為 1)
# if isinstance(shap_values, list) and len(shap_values) == 2:
#     shap_values_for_class_1 = shap_values[1]
# else:
#     shap_values_for_class_1 = shap_values # 如果模型輸出只有一維 (如機率)

# # SHAP 摘要圖
# print("\nSHAP Summary Plot (請在新視窗中查看):")
# shap.summary_plot(shap_values_for_class_1, X)

# # SHAP 條形圖 (平均絕對 SHAP 值)
# print("\nSHAP Bar Plot (平均絕對 SHAP 值):")
# shap.summary_plot(shap_values_for_class_1, X, plot_type="bar")

# # 單個預測的 SHAP Waterfall Plot (例如，第一個測試樣本)
# # print("\nSHAP Waterfall Plot for a single prediction (第一個測試樣本):")
# # shap.plots.waterfall(shap.Explanation(
# #     values=shap_values_for_class_1[0],
# #     base_values=explainer.expected_value[1] if isinstance(explainer.expected_value, list) else explainer.expected_value,
# #     data=X.iloc[0].values,
# #     feature_names=X.columns.tolist()
# # ))

# # SHAP Dependence Plot (例如，針對 'language_professionalism_score')
# # print("\nSHAP Dependence Plot for 'language_professionalism_score':")
# # shap.dependence_plot("language_professionalism_score", shap_values_for_class_1, X, interaction_index=None)

開始訓練 XGBoost 模型...


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [08:47:08] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


模型訓練完成。

模型評估結果 (測試集):
準確度 (Accuracy): 0.8350
精確度 (Precision): 0.8350
召回率 (Recall): 1.0000
F1 分數 (F1 Score): 0.9101
ROC AUC: 0.8220

特徵重要性:
                  Feature  Importance
2           nb_hyperlinks    0.652032
9          ratio_extMedia    0.083861
7        external_favicon    0.054027
15        has_js_redirect    0.040725
8           links_in_tags    0.037832
3      ratio_intHyperlink    0.034381
6         ratio_extErrors    0.033594
4     ratio_extHyperlinks    0.010983
14       has_meta_refresh    0.009562
1         domain_in_brand    0.009021
11            empty_title    0.008807
10            safe_anchor    0.006355
0             phish_hints    0.005506
12        domain_in_title    0.005186
13  domain_with_copyright    0.004534
5    ratio_extRedirection    0.003595


In [ ]:
model.save_model("url_xgb_v1.json")